# Generating Populations and Species
Assuming that the civilization is on the verge of moving from a phaze 0 to phase 1 civ.
* baseline species
* n populations (pops)
* population varience (within the norms of the species)
* allegiance groups to factions

Starting with an example user form data

In [1]:

from numpy import random, round, interp, linspace
import pickle
from sklearn.cluster import KMeans
import altair as alt
import sys, os
from sklearn.cluster import KMeans

import pandas as pd


origional `data` comes from the user via the form in `forms.py`

In [2]:
starting_attributes = ['conformity', 'literacy', 'aggression', 'constitution']

data = {
    'conformity':0.4,
    'literacy':0.7,
    'aggression':0.5,
    'constitution':0.4,
    'starting_pop': 100
}

def build_species(data):
    species = {}
    for attr in starting_attributes:
        species[attr] = data[attr]
    return species
    
species = build_species(data)
species


{'conformity': 0.4, 'literacy': 0.7, 'aggression': 0.5, 'constitution': 0.4}

In [3]:
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f

## Creating groups of populations, all with different attributes

Populations vary based on conformity, some populations are more literate, some are more aggressive. 

In [4]:
pop_std = .2 * (1-species['conformity'])
print("the population conformity: ", pop_std)

def vary_pops(species):
    pop = {}
    for k in list(species.keys()):
        pop[k] = abs(round(random.normal(species[k], pop_std),3))
    return pop

pops = pd.DataFrame([vary_pops(species) for i in range(data['starting_pop'])])
pops

the population conformity:  0.12


,conformity,literacy,aggression,constitution
0,0.479,0.755,0.700,0.578
1,0.478,0.647,0.449,0.196
2,0.377,0.729,0.550,0.211
3,0.230,0.458,0.492,0.621
4,0.414,0.815,0.833,0.361
...,...,...,...,...
95,0.330,0.842,0.325,0.324
96,0.564,0.632,0.603,0.463
97,0.494,0.718,0.432,0.339
98,0.282,0.756,0.366,0.439


The end result is that population factions are grouped by the idealogical differences of the people in them. This gives each faction a distinct culture. 

## Dividing pops into factions

I feel like there isn't a way to pick an ideal number of factions procedurally,  so until I think of something better I'm just going to go with an arbitrary range based on `population_conformity`.

The amount of different nations is relative to `1-population_conformity` and then scaled out over a number of steps defined as `n_steps`

In [5]:
n_steps = 6

def get_n_factions(n_steps):
    x = interp(
        (1-data["conformity"]),
            linspace(0, 1, num=n_steps),
            [i for i in range(n_steps)]
        )
    return int(round(x))

get_n_factions(n_steps)

3

In [6]:
kmeans = KMeans(n_clusters=n_steps).fit(pops[starting_attributes])
pops['faction_no'] = kmeans.labels_

In [7]:
factions = pd.DataFrame(kmeans.cluster_centers_, columns=starting_attributes)
factions['name'] = factions['conformity'].apply(lambda x: f.make_word(1))
factions

,conformity,literacy,aggression,constitution,name
0,0.398241,0.710103,0.543793,0.280897,Mia
1,0.315056,0.767111,0.419667,0.494556,Greens
2,0.507563,0.675063,0.377188,0.451250,Chaic
3,0.390231,0.919846,0.647462,0.420846,Weyn
4,0.350000,0.513818,0.469000,0.427091,Trubc
5,0.515846,0.671615,0.634385,0.480154,Pi


So individual `pops` vary slightly, relative to the `conformity`. And `conformity` also determines the number of factions in the group. 

In [8]:
chart = alt.Chart(pops).mark_circle().encode(x='literacy',y='aggression',color='faction_no:N')
chart

alt.Chart(...)

The cluster centers can represent the 'zeitgeist' of that faction's culture. 

# Other population attributes

## Other attributes that get added
These items aren't used to calculate factions as they refer to resources, not values. These are calculated at the creation of a `pop` and later can be updated. 

* `industry` = `aggression` + `constitution`
* `wealth` = sum(`literacy` + `industry`)/2



In [16]:
chart = alt.Chart(pops).mark_circle().encode(x='wealth',y='industry',color='faction_no:N')
chart

alt.Chart(...)

In [9]:
pops['industry'] = (pops['aggression'] + pops['constitution'])/2
pops['wealth'] = (pops['literacy'] + pops['industry'])/2
pops['target_score'] = (1-pops['conformity']) - pops['aggression']
pops

,conformity,literacy,aggression,constitution,faction_no,industry,wealth,target_score
0,0.479,0.755,0.700,0.578,5,0.6390,0.69700,-0.179
1,0.478,0.647,0.449,0.196,0,0.3225,0.48475,0.073
2,0.377,0.729,0.550,0.211,0,0.3805,0.55475,0.073
3,0.230,0.458,0.492,0.621,4,0.5565,0.50725,0.278
4,0.414,0.815,0.833,0.361,3,0.5970,0.70600,-0.247
...,...,...,...,...,...,...,...,...
95,0.330,0.842,0.325,0.324,1,0.3245,0.58325,0.345
96,0.564,0.632,0.603,0.463,5,0.5330,0.58250,-0.167
97,0.494,0.718,0.432,0.339,2,0.3855,0.55175,0.074
98,0.282,0.756,0.366,0.439,1,0.4025,0.57925,0.352


In [17]:
chart = alt.Chart(
        pops.reset_index(drop=False)
        ).mark_bar(size=4).encode(
            x=alt.X('index:N', sort='-y'),
            y='wealth',
            color='faction_no:N').properties(width=600,title='Some populations make better targets than others')
chart

alt.Chart(...)

In [55]:
chart.encode(
            x=alt.X('faction_no:N', sort='-y'),
            y='wealth',
        
            color='faction_no:N').properties(title='Some factions have greater wealth')

alt.Chart(...)

In [18]:
# chart_pivot = pops.unstack().reset_index(drop=False).rename(columns={'level_0':'attribute', 'level_1':'item', 0:'value'})
# chart_pivot

# Faction Loyalty
* More aggressive pops will have less loyalty. 
* populations with values more distant to the faction will have less loyalty. 

How do you calculate the difference between values of two groups?

In [13]:
def compare_values(values,gr1,gr2):
    distance = []
    for v in values:
        distance.append(abs(gr1.get(v,0)-gr2.get(v,0)))
    return sum(distance)/len(distance)

compare_values(starting_attributes,
    pops.loc[0].to_dict(),
    pops.loc[2].to_dict())

0.16124999999999998

The average of the distance between values indicates the idealogical separation between two peoples. The same can be used to measure a people and it's faction, as well as the animosity between two factions. 

In [59]:
def get_faction_loyalty(x):
    g1 = pops.loc[x].to_dict()
    f2 = factions.loc[int(g1['faction_no'])].to_dict()
    return compare_values(starting_attributes,g1,f2)


pops['faction_loyalty'] = [get_faction_loyalty(i) for i in pops.index]

alt.Chart(pops).mark_point().encode(
            x=alt.X('aggression:Q', sort='-y'),
            y='faction_loyalty:Q',
            color='faction_no:N').properties(title='some factions make better targets as well')


alt.Chart(...)